In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("four").getOrCreate()

In [9]:
from pyspark.sql.functions import regexp_replace,explode,split

In [48]:
df1 = spark.read.text('DataSets/reg_1.txt')
df1.show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                       |
+------------------------------------------------------------------------------------------------------------------------------------------------------------+
|RechargeId|Rechargedate|Remaining_days|validity|R201623|20200511|1|online|R201873|20200119|110|online|R201999|20200105|35|online|R201951|20191105|215|online|
+------------------------------------------------------------------------------------------------------------------------------------------------------------+



In [49]:
df2 = df1.withColumn("NewCol",regexp_replace(df1.value,"(.*?\\|){4}","$0=")).drop('value')
df2.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|NewCol                                                                                                                                                          |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|RechargeId|Rechargedate|Remaining_days|validity|=R201623|20200511|1|online|=R201873|20200119|110|online|=R201999|20200105|35|online|=R201951|20191105|215|online|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------+



In [56]:
df3 = df2.withColumn("Exp",explode(split(df2.NewCol,'\|='))).drop('NewCol')
df3.show(truncate=False)

+-----------------------------------------------+
|Exp                                            |
+-----------------------------------------------+
|RechargeId|Rechargedate|Remaining_days|validity|
|R201623|20200511|1|online                      |
|R201873|20200119|110|online                    |
|R201999|20200105|35|online                     |
|R201951|20191105|215|online                    |
+-----------------------------------------------+



In [73]:
df4 = df3.withColumn("Splitted",split(df3.Exp,'\|')) \
        .withColumn("One",split(df3.Exp,'\|')[0]) \
        .withColumn("Two",split(df3.Exp,'\|')[1]) \
        .withColumn("Three",split(df3.Exp,'\|')[2])\
        .withColumn("Four",split(df3.Exp,'\|')[3]) \
        .drop('Exp')
df4.show(truncate=False)

+----------------------------------------------------+----------+------------+--------------+--------+
|Splitted                                            |One       |Two         |Three         |Four    |
+----------------------------------------------------+----------+------------+--------------+--------+
|[RechargeId, Rechargedate, Remaining_days, validity]|RechargeId|Rechargedate|Remaining_days|validity|
|[R201623, 20200511, 1, online]                      |R201623   |20200511    |1             |online  |
|[R201873, 20200119, 110, online]                    |R201873   |20200119    |110           |online  |
|[R201999, 20200105, 35, online]                     |R201999   |20200105    |35            |online  |
|[R201951, 20191105, 215, online]                    |R201951   |20191105    |215           |online  |
+----------------------------------------------------+----------+------------+--------------+--------+

